## unstack과 pivot을 사용해 데이터를 넓은 포멧으로 리셰이핑
- 타이디한 데이터를 그렇지 못하게 바꿔야할 때도 있다 -> 관계형 데이터를 잘 다루지 못할 때, 정해진 포맷에 맞춰야 할 때
- stack <-> unstack / melt <-> pivot

In [1]:
import pandas as pd
 
nls97=pd.read_csv('C:/data-cleansing-main/Chapter09/data/nls97f.csv')
nls97.set_index('originalid',inplace=True)

In [2]:
weeksworkedcols = ['weeksworked00','weeksworked01','weeksworked02','weeksworked03','weeksworked04']
weeksworkedstacked=nls97[weeksworkedcols].stack(dropna=False)
weeksworkedstacked.loc[[1,2]]

originalid               
1           weeksworked00    53.0
            weeksworked01    52.0
            weeksworked02     NaN
            weeksworked03    42.0
            weeksworked04    52.0
2           weeksworked00    51.0
            weeksworked01    52.0
            weeksworked02    44.0
            weeksworked03    45.0
            weeksworked04    52.0
dtype: float64

In [4]:
# 데이터 녹이기
weeksworkedmelted=nls97.reset_index().loc[:,['originalid']+weeksworkedcols].\
    melt(id_vars=['originalid'],value_vars=weeksworkedcols,var_name='year',value_name='weeksworked')

weeksworkedmelted.loc[weeksworkedmelted.originalid.isin([1,2])].sort_values(['originalid','year'])

,originalid,year,weeksworked
377,1,weeksworked00,53.0
9361,1,weeksworked01,52.0
18345,1,weeksworked02,NaN
27329,1,weeksworked03,42.0
36313,1,weeksworked04,52.0
8980,2,weeksworked00,51.0
17964,2,weeksworked01,52.0
26948,2,weeksworked02,44.0
35932,2,weeksworked03,45.0
44916,2,weeksworked04,52.0


### unstack 사용

In [5]:
weeksworked=weeksworkedstacked.unstack()
weeksworked.loc[[1,2]]

,weeksworked00,weeksworked01,weeksworked02,weeksworked03,weeksworked04
originalid,,,,,
1,53.0,52.0,NaN,42.0,52.0
2,51.0,52.0,44.0,45.0,52.0


### pivot 사용
1. index: 인덱스 열
2. columns: 값을 열의 이름으로 추가할 열
3. values: 넓은 포맷으로 되돌릴 열들의 이름

In [20]:
weeksworked=weeksworkedmelted.pivot(index='originalid',columns='year',values=['weeksworked']).reset_index()

weeksworked.columns=['originalid']+[col[1] for col in weeksworked.columns[1:]]     # 그 결과로 다중 수준의 열 이름 반환, 두 번째를 꺼냄으로 이를 수정할 수 있음
weeksworked.loc[weeksworked.originalid.isin([1,2])]

,originalid,weeksworked00,weeksworked01,weeksworked02,weeksworked03,weeksworked04
0,1,53.0,52.0,NaN,42.0,52.0
1,2,51.0,52.0,44.0,45.0,52.0
